In [1]:
ACTION = {'UP':[0,0,1],'DOWN':[0,0,-1], 'NORTH':[1,0,0], 'SOUTH':[-1,0,0], 'WEST':[0,1,0], 'EAST':[0,-1,0]}

In [51]:
from random import randint
class Agent:
    
    x = 0; 
    y = 0;
    z = 0;
    x_lim = 0;
    y_lim = 0;
    z_lim = 0;
    rank = 1;
    
    def __init__(self,x_lim,y_lim,z_lim):
        self.x_lim = x_lim;
        self.y_lim = y_lim;
        self.z_lim = z_lim;
        self.__init_rand_state();
        
    def __init_rand_state(self):
        self.x = randint(0, self.x_lim);
        self.y = randint(0, self.y_lim);
        self.z = randint(0, self.z_lim);
        
    def get_state(self):
        return [self.x, self.y, self.z];
    
    def set_state(self, x, y, z):
        if x <= self.x_lim & y <= self.y_lim & z <= self.z_lim:
            self.x = x;
            self.y = y;
            self.z = z;
        
    #need to set boundary 
    def move(self,act):
        act_value = ACTION.get(act);
        self.set_state(self.x+act_value[0],self.y+act_value[1],self.z+act_value[2]);
        
    

In [52]:
a1 = Agent(7,7,4);
print(a1.x);
print(a1.y);
print(a1.z);

3
6
0


In [54]:
a1.move('UP')
print(a1.get_state())

[3, 6, 0]


In [50]:
#1.set_state(1,1,1)

a1.move('NORTH')
a1.get_state()

[4, 7, 2]

In [59]:
x = 1
y = 2
0<=x&y<=3


True